In [7]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [9]:
subject = 'Piauí - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [10]:
data = pd.read_csv('2003_01_model_input_PI.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH,Piauí - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,317.286319,8.192266,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,0.649218,238.886000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,351.667485,8.118950,2.172536e+07,1.379534e+06,6.108224,1.884442e+07,0.654538,276.711000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,357.819089,8.098198,2.207657e+07,1.395739e+06,6.154515,1.899957e+07,0.659288,272.764000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,386.232189,8.006233,2.240750e+07,1.409503e+06,6.196868,1.913991e+07,0.663482,333.178000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,445.515113,7.980382,2.268410e+07,1.390317e+06,6.221036,1.922183e+07,0.667129,386.585000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,510.644488,7.951466,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,0.670241,449.076000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,564.654167,7.981121,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,0.672829,536.766000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,595.360558,7.946167,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,0.674904,639.535000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,612.063590,7.934312,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,0.676479,625.022000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,616.463496,9.098665,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,0.677105,687.536000


In [11]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,317.286319,8.192266,2.136063e+07,1.358044e+06,6.061141,1.868149e+07,0.649218
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,351.667485,8.118950,2.172536e+07,1.379534e+06,6.108224,1.884442e+07,0.654538
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,357.819089,8.098198,2.207657e+07,1.395739e+06,6.154515,1.899957e+07,0.659288
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,386.232189,8.006233,2.240750e+07,1.409503e+06,6.196868,1.913991e+07,0.663482
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,445.515113,7.980382,2.268410e+07,1.390317e+06,6.221036,1.922183e+07,0.667129
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,510.644488,7.951466,2.296156e+07,1.400359e+06,6.253673,1.932330e+07,0.670241
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,564.654167,7.981121,2.318896e+07,1.418401e+06,6.277339,1.939333e+07,0.672829
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,595.360558,7.946167,2.385426e+07,1.441334e+06,6.336644,1.960635e+07,0.674904
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,612.063590,7.934312,2.447423e+07,1.448568e+06,6.385667,1.978514e+07,0.676479
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,616.463496,9.098665,2.502337e+07,1.450923e+06,6.419744,1.991803e+07,0.677105


In [12]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     276.711000
1     272.764000
2     333.178000
3     386.585000
4     449.076000
5     536.766000
6     639.535000
7     625.022000
8     687.536000
9     778.779000
10    755.713857
11    794.945486
12    842.687657
13    831.356000
14    626.223000
15    702.437000
16    788.164000
17    826.368000
18    804.417000
19           NaN
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [13]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.870412,-0.759482,-1.609767,-2.279691,-1.909036,-1.868082,-2.083911
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.618988,-0.812606,-1.411497,-1.527027,-1.587774,-1.553382,-1.636049
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.574002,-0.827643,-1.220572,-0.959475,-1.271914,-1.253703,-1.236089
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.366222,-0.894281,-1.040671,-0.477405,-0.982926,-0.982635,-0.883077
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,0.067305,-0.913012,-0.890309,-1.149354,-0.818020,-0.824410,-0.576060
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,0.543586,-0.933964,-0.739477,-0.797670,-0.595326,-0.628420,-0.314085
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,0.938551,-0.912477,-0.615859,-0.165758,-0.433844,-0.493156,-0.096198
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,1.163102,-0.937804,-0.254194,0.637432,-0.029186,-0.081704,0.078553
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,1.285248,-0.946394,0.082831,0.890796,0.305317,0.263623,0.211121
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,1.317424,-0.102709,0.381351,0.973286,0.537836,0.520297,0.263836


In [14]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     276.711000
1     272.764000
2     333.178000
3     386.585000
4     449.076000
5     536.766000
6     639.535000
7     625.022000
8     687.536000
9     778.779000
10    755.713857
11    794.945486
12    842.687657
13    831.356000
14    626.223000
15    702.437000
16    788.164000
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.746865,1.11282,0.859906,0.717584,0.670939,0.684873,1.034736


In [16]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    826.368
Name: Piauí - Consumo de Cimento (t), dtype: float64

In [17]:
train_input.iloc[-(len(train_input)//5):]

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
14,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,-1.185030,1.400373,1.231996,0.363973,1.101013,1.123692,1.337504
15,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,-1.621324,1.302001,1.223953,0.423683,1.075324,1.095446,1.305811
16,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-1.859259,1.286548,1.108414,0.590900,0.919316,0.939188,1.232971


In [18]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [22]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1291080699, 1459397082, 1549140460, 1168471884, 3878284011, 2020136445, 2129136314, 3805880403, 1028626661, 4065827638, 2613447730, 3651072523, 2188936883, 1272647764, 1846138695, 3535907355, 4109027244, 835896312, 1481475622, 4261701291, 182906959, 2455400977, 2672382868, 1684675641, 248081064, 1438083547, 4089337478, 3064659757, 1248737576, 562961575, 2508702561, 4253826545, 2095723673, 619914785, 2808870205, 3736253010, 222075022, 2989883636, 3793586250, 2180413161, 3692718794, 2177002007, 3420137684, 2172068199, 2515297515, 1359004156, 1676077991, 3684234764, 2865658532, 889335743, 3603894599, 533406578, 3073342084, 4014146477, 915698224, 1090947652, 2670677172, 165301316, 2017735677, 3410907217, 1976886848, 3206270738, 4204875668, 3832217705, 1052197192, 1309244619, 1975733369, 2222619176, 1429637379, 1447395930, 759019919, 953481524, 3864835565, 1682688856, 2998282730, 284693852, 163757180, 3690277456, 3336938305, 614296521, 1290034546, 4202511165, 3939520629, 3881035940, 149456

val_loss: 6084.62353515625


Step: 87 ___________________________________________
val_loss: 7116.279296875


Step: 88 ___________________________________________
val_loss: 4573.81884765625


Step: 89 ___________________________________________
val_loss: 4958.01123046875


Step: 90 ___________________________________________
val_loss: 4389.873046875


Step: 91 ___________________________________________
val_loss: 4375.859375


Step: 92 ___________________________________________
val_loss: 3257.9580078125


Step: 93 ___________________________________________
val_loss: 4376.05419921875


Step: 94 ___________________________________________
val_loss: 4496.0791015625


Step: 95 ___________________________________________
val_loss: 6593.84814453125


Step: 96 ___________________________________________
val_loss: 4719.47509765625


Step: 97 ___________________________________________
val_loss: 4376.76953125


Step: 98 ___________________________________________
val_loss: 7411.447265625


Ste

In [23]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 436ms/step - loss: 405279.1562 - val_loss: 473281.1562
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 385358.0312 - val_loss: 132621.7656
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 406271.4688 - val_loss: 258891.6719
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 347027.4375 - val_loss: 271592.5625
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 234854.9531 - val_loss: 66750.9766
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 228911.8281 - val_loss: 421754.7500
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 246394.0625 - val_loss: 169523.2969
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 205037.6875 - val_loss: 198686.4531
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 191516.6094 - val_loss: 160490.3594
Epoch 10/1

1/1 [==============================] - 0s 13ms/step - loss: 71995.9844 - val_loss: 11861.1064
Epoch 76/10000
1/1 [==============================] - 0s 15ms/step - loss: 70838.4297 - val_loss: 11641.8584
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 70918.0078 - val_loss: 11080.2295
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 68564.0703 - val_loss: 11366.2490
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 67009.6250 - val_loss: 8614.9043
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 67262.7656 - val_loss: 10702.0400
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 66026.1797 - val_loss: 10047.0908
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 69174.7969 - val_loss: 22200.2266
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 70773.4922 - val_loss: 4853.2739
Epoch 84/10000
1/1 [================

1/1 [==============================] - 0s 13ms/step - loss: 24454.4902 - val_loss: 7750.2637
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 24203.1328 - val_loss: 7633.4624
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 22996.1602 - val_loss: 6986.6719
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 23320.9238 - val_loss: 7434.9546
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 22966.4141 - val_loss: 7313.6235
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 22342.1426 - val_loss: 7310.2402
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 22760.7441 - val_loss: 7769.7085
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 22127.2559 - val_loss: 7571.0630
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 22180.7773 - val_loss: 7635.4946
Epoch 159/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 11547.5908 - val_loss: 796.6729
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 11481.0283 - val_loss: 811.4006
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 11066.4355 - val_loss: 886.9468
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 11489.7070 - val_loss: 854.2116
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 11170.0234 - val_loss: 859.6850
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 10985.9102 - val_loss: 873.2742
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 11075.4688 - val_loss: 896.1657
Epoch 232/10000
1/1 [==============================] - 0s 13ms/step - loss: 10881.6992 - val_loss: 900.0611
Epoch 233/10000
1/1 [==============================] - 0s 13ms/step - loss: 10724.1680 - val_loss: 911.1521
Epoch 234/10000
1/1 [=======================

1/1 [==============================] - 0s 13ms/step - loss: 6991.8066 - val_loss: 2070.4978
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 7071.3975 - val_loss: 1859.4852
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 6747.3545 - val_loss: 2102.2649
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 6992.3604 - val_loss: 1894.9769
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 7201.9546 - val_loss: 2063.6067
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 6726.5322 - val_loss: 1985.9249
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 7045.5654 - val_loss: 2048.6299
Epoch 308/10000
1/1 [==============================] - 0s 14ms/step - loss: 6857.3257 - val_loss: 1906.4073
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 6693.0859 - val_loss: 2426.0891
Epoch 310/10000
1/1 [=======================

...

1/1 [==============================] - 0s 14ms/step - loss: 4409.5908 - val_loss: 5059.9526
Epoch 834/10000
1/1 [==============================] - 0s 14ms/step - loss: 4406.2373 - val_loss: 5065.8184
Epoch 835/10000
1/1 [==============================] - 0s 13ms/step - loss: 4402.9282 - val_loss: 5071.8105
Epoch 836/10000
1/1 [==============================] - 0s 13ms/step - loss: 4399.6621 - val_loss: 5077.9287
Epoch 837/10000
1/1 [==============================] - 0s 14ms/step - loss: 4396.4385 - val_loss: 5084.1357
Epoch 838/10000
1/1 [==============================] - 0s 14ms/step - loss: 5503.8115 - val_loss: 5839.3384
Epoch 839/10000
1/1 [==============================] - 0s 13ms/step - loss: 4441.0386 - val_loss: 4931.0308
Epoch 840/10000
1/1 [==============================] - 0s 13ms/step - loss: 4392.5640 - val_loss: 5136.0454
Epoch 841/10000
1/1 [==============================] - 0s 14ms/step - loss: 4387.0913 - val_loss: 5096.6372
Epoch 842/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 4239.7637 - val_loss: 5576.4858
Epoch 910/10000
1/1 [==============================] - 0s 14ms/step - loss: 4238.2012 - val_loss: 5583.0376
Epoch 911/10000
1/1 [==============================] - 0s 14ms/step - loss: 4236.6528 - val_loss: 5589.5698
Epoch 912/10000
1/1 [==============================] - 0s 13ms/step - loss: 4235.1211 - val_loss: 5596.0942
Epoch 913/10000
1/1 [==============================] - 0s 13ms/step - loss: 4233.6025 - val_loss: 5602.5991
Epoch 914/10000
1/1 [==============================] - 0s 14ms/step - loss: 4231.0083 - val_loss: 5609.7993
Epoch 915/10000
1/1 [==============================] - 0s 13ms/step - loss: 4230.6104 - val_loss: 5615.3989
Epoch 916/10000
1/1 [==============================] - 0s 13ms/step - loss: 4229.1348 - val_loss: 5622.0430
Epoch 917/10000
1/1 [==============================] - 0s 14ms/step - loss: 4227.6714 - val_loss: 5628.4512
Epoch 918/10000
1/1 [=======================

1/1 [==============================] - 0s 14ms/step - loss: 4239.0791 - val_loss: 5225.3853
Epoch 986/10000
1/1 [==============================] - 0s 13ms/step - loss: 4226.3901 - val_loss: 5145.6650
Epoch 987/10000
1/1 [==============================] - 0s 13ms/step - loss: 4223.1157 - val_loss: 5177.2446
Epoch 988/10000
1/1 [==============================] - 0s 13ms/step - loss: 3762.4392 - val_loss: 4679.5640
Epoch 989/10000
1/1 [==============================] - 0s 14ms/step - loss: 3891.7634 - val_loss: 4933.9390
Epoch 990/10000
1/1 [==============================] - 0s 13ms/step - loss: 4116.6587 - val_loss: 5047.5830
Epoch 991/10000
1/1 [==============================] - 0s 13ms/step - loss: 3998.7490 - val_loss: 5194.3774
Epoch 992/10000
1/1 [==============================] - 0s 13ms/step - loss: 4115.6655 - val_loss: 5002.4697
Epoch 993/10000
1/1 [==============================] - 0s 14ms/step - loss: 4119.1455 - val_loss: 5222.1899
Epoch 994/10000
1/1 [=======================

1/1 [==============================] - 0s 13ms/step - loss: 4057.1213 - val_loss: 5454.2866
Epoch 1061/10000
1/1 [==============================] - 0s 13ms/step - loss: 4055.4702 - val_loss: 5646.9434
Epoch 1062/10000
1/1 [==============================] - 0s 13ms/step - loss: 3874.2161 - val_loss: 5304.6401
Epoch 1063/10000
1/1 [==============================] - 0s 14ms/step - loss: 4144.3633 - val_loss: 5683.2109
Epoch 1064/10000
1/1 [==============================] - 0s 13ms/step - loss: 3874.0854 - val_loss: 5296.8853
Epoch 1065/10000
1/1 [==============================] - 0s 14ms/step - loss: 4144.2622 - val_loss: 5686.7642
Epoch 1066/10000
1/1 [==============================] - 0s 13ms/step - loss: 3952.7781 - val_loss: 5448.1787
Epoch 1067/10000
1/1 [==============================] - 0s 13ms/step - loss: 4052.6812 - val_loss: 5654.9038
Epoch 1068/10000
1/1 [==============================] - 0s 13ms/step - loss: 4134.8384 - val_loss: 5619.2339
Epoch 1069/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 3665.8264 - val_loss: 6523.1538
Epoch 1136/10000
1/1 [==============================] - 0s 13ms/step - loss: 3592.7756 - val_loss: 6787.3579
Epoch 1137/10000
1/1 [==============================] - 0s 13ms/step - loss: 3336.0198 - val_loss: 6729.8843
Epoch 1138/10000
1/1 [==============================] - 0s 13ms/step - loss: 4892.5645 - val_loss: 8042.2280
Epoch 1139/10000
1/1 [==============================] - 0s 13ms/step - loss: 3233.0820 - val_loss: 6699.3882
Epoch 1140/10000
1/1 [==============================] - 0s 13ms/step - loss: 3707.7998 - val_loss: 6704.3364
Epoch 1141/10000
1/1 [==============================] - 0s 14ms/step - loss: 3541.2451 - val_loss: 6670.8359
Epoch 1142/10000
1/1 [==============================] - 0s 13ms/step - loss: 3520.7307 - val_loss: 6698.4136
Epoch 1143/10000
1/1 [==============================] - 0s 13ms/step - loss: 4912.6050 - val_loss: 5209.3511
Epoch 1144/10000
1/1 [==============

In [24]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[826.368](test_target) - [[785.1676]](prediction) = 41.20039746093755


In [25]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [26]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.260938,1.865075,-1.539675,-1.748395,-1.648252,-1.663210,-1.736497
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.910929,0.999186,-1.060422,-0.834425,-1.094089,-1.086381,-1.109588
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.848304,0.754096,-0.598923,-0.145240,-0.549245,-0.537085,-0.549730
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.559051,-0.332049,-0.164072,0.440144,-0.050753,-0.040230,-0.055589
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.044464,-0.637362,0.199378,-0.375812,0.233702,0.249788,0.374169
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.707498,-0.978865,0.563966,0.051243,0.617841,0.609028,0.740877
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.257330,-0.628630,0.862772,0.818581,0.896389,0.856960,1.045872
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.569929,-1.041451,1.736977,1.793905,1.594406,1.611130,1.290485


train_target:


,Piauí - Consumo de Cimento (t)
0,276.711
1,272.764
2,333.178
3,386.585
4,449.076
5,536.766
6,639.535
7,625.022


test_input:


...

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Piauí - Produção de Cimento (t),Piauí - Desemprego,Piauí - PIB - Estadual,Piauí - PIB - Construção Civil,Piauí - PIB - Per Capita,Piauí - PIB - Preços de Mercado,Piauí - IDH
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-1.746865,1.11282,0.859906,0.717584,0.670939,0.684873,1.034736


test_target:


,Piauí - Consumo de Cimento (t)
17,826.368


1/1 [==============================] - 0s 34ms/step
Error: 41.20039746093755




[563.5429077148438,
 624.4970092773438,
 761.3983764648438,
 708.587646484375,
 763.3076171875,
 811.3246459960938,
 812.8284301757812,
 759.6661987304688,
 704.2880859375,
 785.1676025390625]

In [27]:
display(targets)
display(predictions)

[687.536,
 778.779,
 755.7138571428571,
 794.9454857142857,
 842.6876571428572,
 831.356,
 626.223,
 702.437,
 788.164,
 826.368]

[563.5429077148438,
 624.4970092773438,
 761.3983764648438,
 708.587646484375,
 763.3076171875,
 811.3246459960938,
 812.8284301757812,
 759.6661987304688,
 704.2880859375,
 785.1676025390625]

In [28]:
mae = mean_absolute_error(predictions, targets)
mae

83.86397759486609

In [29]:
porcentage = mae/np.mean(targets)
porcentage

0.10985285654293775